In [1]:
import pandas as pd
import json
import os
import numpy as np

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
#import spacy
import nltk
nltk.download('stopwords')

# spacy for lemmatization
import spacy
nlp = spacy.load('en', disable=['parser', 'ner'])

C:\Users\TUSHAR\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TUSHAR\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# tokenize
def tokenize_words(text):
    token = []
    count=0
    lemmat_tokens=[]
    wordnet_lemmatizer = WordNetLemmatizer()
    total = len(text)
    #print((total))    
    try:        
        for i in range(total):
            count=count+1
            if(text[i] == '' or pd.isnull(text[i])):
                #print("empty str")
                num=9
            else:
                swtoken=[]
                swrtoken=[]
                swtoken=gensim.utils.simple_preprocess(text[i],deacc=True)
                token.append(swtoken)
                    
    except:
        print("text = ",text[i])
    
    return token 

In [3]:
path = 'C:\\Users\\TUSHAR\\Desktop\\unis_MSD\\capstone project\\notion\\dataset\\R4'
os.chdir(path)

data_df=pd.read_csv('article_content.csv', encoding='utf-8')
summary_text=list(data_df['content'])

from nltk.corpus import stopwords
stopWords = stopwords.words('english')
#stopWords = list(stopwords.words('english'))
stopWords.extend(["commission","royal","said","australia","let","told","also","ms","com","www","https","au","us","would","mailto","subject","body","says","orr","acbf","mr"]) 

#tokens = tokenize_words(summary_text)
data_words = tokenize_words(summary_text)

In [41]:
#print(data_words)

In [4]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=2, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

C:\Users\TUSHAR\Anaconda3\lib\site-packages\gensim\models\phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['by', 'virtue', 'of', 'its', 'legislation', 'victoria', 'already', 'has', 'comparatively', 'weak', 'anti_corruption', 'body', 'but', 'thanks', 'to', 'bill_introduced', 'very', 'recently', 'into', 'state', 'parliament', 'it', 'is', 'about', 'to', 'become', 'even', 'weaker', 'as', 'its', 'ability', 'to', 'hold', 'public', 'hearings', 'is', 'to', 'be', 'severely', 'restricted', 'restricting', 'ibac', 'ability', 'to', 'instigate', 'public', 'hearings', 'is', 'giant', 'leap', 'backward', 'for', 'victoria', 'and', 'could', 'result', 'in', 'the', 'corrupt', 'conduct', 'of', 'some', 'public', 'servants', 'elected', 'and', 'appointed', 'being', 'protected', 'from', 'the', 'public', 'gaze', 'for', 'many', 'years', 'longer', 'than', 'it', 'should', 'be', 'while', 'it', 'is', 'not', 'possible', 'to', 'know', 'how', 'long', 'the', 'extensive', 'and', 'widespread', 'malfeasance', 'would', 'have', 'continued', 'what', 'is', 'unarguable', 'is', 'that', 'the', 'public', 'hearings', 'laid_bare', 'to', 

In [5]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopWords] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [6]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
data_words_trigrams = make_trigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['virtue', 'legislation', 'victoria', 'already', 'comparatively', 'weak', 'anti_corruption', 'thank', 'bill_introduc', 'recently', 'state', 'parliament', 'become', 'even', 'weak', 'ability', 'hold', 'public', 'hearing', 'severely', 'restrict', 'restricting', 'ibac', 'ability', 'instigate', 'public', 'hearing', 'giant', 'leap', 'backward', 'victoria', 'could', 'result', 'corrupt', 'conduct', 'public', 'servant', 'elect', 'appoint', 'protect', 'public', 'gaze', 'many', 'year', 'longer', 'possible', 'know', 'long', 'extensive', 'widespread', 'malfeasance', 'continue', 'unarguable', 'public', 'hearing', 'laid_bare', 'queenslander', 'extent', 'hide', 'secret', 'corruption', 'abuse', 'power', 'go', 'year', 'conduct', 'former', 'member', 'parliament', 'eddie', 'obeid', 'minister', 'mine', 'expose', 'taxpayer', 'new', 'south_wale', 'pay', 'salary', 'year', 'state', 'independent', 'corruption', 'decide', 'public', 'interest', 'hold', 'public', 'hearing', 'close', 'home', 'highly', 'doubtful', 

In [43]:
#print(data_lemmatized)

In [7]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2), (1, 1), (2, 1), (3, 2), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 3), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 5), (40, 1), (41, 1), (42, 2), (43, 1), (44, 3), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 9), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 3), (95, 1), (96, 1), (97, 2), (98, 1), (99, 3)]]


In [51]:
print(id2word.id2token)

{0: 'ability', 1: 'abuse', 2: 'accountability', 3: 'already', 4: 'anti_corruption', 5: 'appoint', 6: 'backward', 7: 'ballarat', 8: 'become', 9: 'bill_introduc', 10: 'close', 11: 'commit', 12: 'comparatively', 13: 'conduct', 14: 'consider', 15: 'constrain', 16: 'continue', 17: 'corrupt', 18: 'corruption', 19: 'could', 20: 'decide', 21: 'decision', 22: 'department', 23: 'difficult', 24: 'doubtful', 25: 'eddie', 26: 'education', 27: 'effectiveness', 28: 'elect', 29: 'even', 30: 'expose', 31: 'extensive', 32: 'extent', 33: 'former', 34: 'gaze', 35: 'genuinely', 36: 'giant', 37: 'go', 38: 'government', 39: 'hearing', 40: 'hide', 41: 'highly', 42: 'hold', 43: 'home', 44: 'ibac', 45: 'impede', 46: 'independent', 47: 'instigate', 48: 'interest', 49: 'know', 50: 'laid_bare', 51: 'leap', 52: 'learn', 53: 'legislation', 54: 'long', 55: 'longer', 56: 'make', 57: 'malfeasance', 58: 'many', 59: 'matter', 60: 'member', 61: 'mine', 62: 'minister', 63: 'misconduct', 64: 'new', 65: 'obeid', 66: 'officer

In [54]:
# Build LDA model
lmodel = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=50, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           iterations=50,
                                           alpha='auto',
                                           per_word_topics=True)

In [9]:
#tokens = tokenize_words(content)


In [55]:
ltopics=[]
for i in range(lmodel.num_topics):
    topic_list_n=lmodel.print_topic(i)
    temp_token_1=topic_list_n.split('+')
    for ctoken in temp_token_1:
        temp_token_2=ctoken.split('*')
        temp_token_3 = temp_token_2[1]
        temp_token_4=temp_token_3.split('"')
        ind_topic=temp_token_4[1]
    #print(itop)
        ltopics.append(np.str(ind_topic))
    
print(ltopics)
unique_topic_list=list(set(ltopics))
print(unique_topic_list)
    


['smh', 'june', 'html', 'news_story', 'theaustralian', 'sunshine_coast', 'june_pm', 'world_show_subsections_world', 'cup_show_subsection', 'new', 'bank', 'financial', 'banking', 'service', 'officer', 'include', 'need', 'chief', 'say', 'superannuation', 'think', 'decade', 'good', 'australian', 'believe', 'plan', 'word', 'work', 'brand', 'multiple', 'spike', 'offer', 'ask', 'blame', 'howden', 'exclusion', 'work', 'payday', 'practice', 'study', 'material', 'emerge', 'affect', 'set', 'wealth_management', 'influence', 'many', 'chief_executive', 'smsf', 'smsfs', 'walsh', 'pay', 'woman', 'family', 'aboriginal', 'child', 'aboriginal_cooperative', 'victoria_believ', 'tracey_walsh', 'take', 'package', 'policy', 'roll', 'social', 'country', 'change', 'increasingly', 'day', 'response', 'parliament', 'business', 'financial', 'adviser', 'wealth', 'group', 'mortgage_brok', 'spin', 'commission', 'overhaul', 'mortgage', 'service', 'failure', 'management', 'financial', 'great', 'take', 'threat', 'positi

['position', 'drive', 'part', 'scandal', 'could', 'nation', 'business', 'study', 'time', 'investigation', 'entity', 'later', 'old', 'count', 'property', 'trust', 'woman', 'commonwealth', 'road', 'think', 'age', 'corporate_regulator', 'rba', 'grandfathered_commission', 'china', 'mortgage_brok', 'misconduct', 'because', 'dealing', 'outline', 'hearing', 'fee', 'june_pm', 'begin', 'produce', 'crisis', 'insurance', 'probe', 'agreement', 'member', 'phone_call', 'past', 'news', 'html', 'steinberg', 'default', 'receive', 'week', 'amount', 'genuine_credit', 'fintech', 'chair', 'smsfs', 'criminal', 'witness', 'deliver', 'bear', 'backlash', 'payday', 'last', 'loan', 'wealth', 'housing', 'important', 'victoria_believ', 'westpac', 'make', 'response', 'case', 'affect', 'submission', 'hold', 'sorry', 'multiple', 'independent', 'become', 'achieve', 'director', 'management', 'rabobank', 'car', 'behaviour', 'consider', 'wealth_management', 'borrower', 'threat', 'gain', 'inflation', 'least', 'editorial',

In [11]:
topics1=['aboriginal','islander','indigenous','mortality','breaches']
#topics=['housing','loans','decreases','consecutive','trend']
topics=unique_topic_list

#tokensl1=tokens
len_docs=len(data_lemmatized)
print(len_docs)
window=100
topic_freq=dict()
co_occur_net=dict()

for t1 in topics:
    for t2 in topics:
        if(t1!= t2):
            #Loop through the docs
            for i in range(len_docs):
                tokensl1=data_lemmatized[i]                
                l=len(tokensl1)
                #Loop through the tokens of a document
                for j in range(l):
                    if(tokensl1[j]==t1):
                        if(j+1+window > l-1):
                            end=l
                        else:
                            end=j+1+window
            
                        str=tokensl1[j+1:end]
                        #print(str)
                        if(t2 in str):
                            #print(t1,t2,"co-occurrence")
                            tkey=t1+':'+t2
                            if(tkey in co_occur_net.keys()):
                                co_freq=co_occur_net[tkey]
                                co_occur_net[tkey]=co_freq+1
                            else:
                                co_occur_net[tkey]=1
                                

711


In [27]:
#len_docs=len(tokens)
topic_freq=dict()

for t1 in topics:
    for i in range(len_docs):
        tokensl1=data_lemmatized[i]
        l=len(tokensl1)
        #Loop through the tokens of a document
        for j in range(l):
            if(tokensl1[j]==t1):
                if(t1 in topic_freq.keys()):
                    topic_freq[t1]=topic_freq[t1]+1
                else:
                    topic_freq[t1]=1
                


print(topic_freq)
print("co_occur")
#print(co_occur_net)

{'hear': 160, 'fund': 309, 'bank': 1212, 'aboriginal': 188, 'policy': 215, 'stock': 85, 'firm': 108, 'advice': 110, 'morningstar': 77, 'director': 77, 'insure': 79, 'business': 429, 'water': 33, 'adviser': 95, 'customer': 306, 'indigenous': 152, 'management': 125, 'labor': 77, 'australian': 375, 'time': 219, 'believe': 80, 'support': 77, 'private': 34, 'campaign': 43, 'credit': 133, 'public': 111, 'news_story': 54, 'new': 333, 'government': 150, 'suffer': 10, 'investor': 200, 'probe': 19, 'per_cent': 378, 'rise': 146, 'property': 202, 'report': 183, 'financial': 688, 'global': 152, 'people': 282, 'sale': 183, 'growth': 263, 'rate': 192, 'fall': 146, 'premium': 40, 'news': 101, 'walsh': 36, 'smh': 74, 'partner': 89, 'banking': 318, 'lending': 71, 'death': 32, 'html': 56, 'big': 160, 'tax': 90, 'make': 221, 'june': 184, 'risk': 186, 'role': 61, 'child': 43, 'company': 252, 'industry': 211, 'service': 308, 'high': 250, 'funeral_insurance': 144, 'pay': 162, 'amount': 42, 'clayton_utz': 52,

In [13]:
edge_weight_conet=dict()

for key,val in co_occur_net.items():
    composite_topics=key.split(':')
    source_topic=composite_topics[0]    
    if(source_topic in topic_freq.keys()):
        ind_freq=topic_freq[source_topic]
        if(ind_freq > 0):
            ew= val/np.int(ind_freq)
            if(ew>0.4):
                edge_weight_conet[key]=ew
    

In [14]:
print(edge_weight_conet)

{'aboriginal:people': 0.526595744680851, 'stock:fund': 0.4117647058823529, 'stock:investor': 0.5647058823529412, 'stock:risk': 0.4470588235294118, 'stock:market': 0.7176470588235294, 'stock:year': 0.5058823529411764, 'advice:financial': 0.5909090909090909, 'morningstar:fund': 0.42857142857142855, 'morningstar:stock': 0.6493506493506493, 'morningstar:director': 0.4155844155844156, 'morningstar:management': 0.5584415584415584, 'morningstar:australian': 0.5194805194805194, 'morningstar:investor': 0.7792207792207793, 'morningstar:rise': 0.4155844155844156, 'morningstar:financial': 0.5194805194805194, 'morningstar:global': 0.4675324675324675, 'morningstar:growth': 0.44155844155844154, 'morningstar:risk': 0.5454545454545454, 'morningstar:market': 0.8311688311688312, 'insure:aboriginal': 0.46835443037974683, 'insure:policy': 0.45569620253164556, 'insure:customer': 0.4177215189873418, 'insure:people': 0.5316455696202531, 'insure:sale': 0.5316455696202531, 'insure:funeral_insurance': 0.46835443

In [15]:
# Make CSV of bigram co-occurrence net
sourceNode=[]
targetNode=[]
edgeWeight=[]

for key,val in edge_weight_conet.items():
    composite_topics=key.split(':')
    sourceNode.append(composite_topics[0])
    targetNode.append(composite_topics[1])
    edgeWeight.append(val)
    
cooccur_df=pd.DataFrame()
cooccur_df['source']=sourceNode
cooccur_df['target']=targetNode
cooccur_df['weight']=edgeWeight

cooccur_df.to_csv('cooccurrence_net.csv',index=False,encoding='utf-8')

In [30]:
# Now work on trigrams:

#edge_weight_conet=dict()
trigram_weight_conet=dict()
stop_words_trigram=['hear', 'business','australian','time','believe','support','public','new','fall','smh','html','make','june','child','year']

for key,val in edge_weight_conet.items():
    composite_topics=key.split(':')
    bi_source_topic=composite_topics[0]    
    bi_target_topic=composite_topics[1]
    if((bi_source_topic not in stop_words_trigram) and (bi_target_topic not in stop_words_trigram)):
        for key2,val2 in edge_weight_conet.items():
            composite_topics2=key2.split(':')
            inner_source_topic=composite_topics2[0]
            inner_target_topic=composite_topics2[1]
            #print(bi_target_topic , inner_source_topic)
            if (bi_target_topic == inner_source_topic and inner_target_topic != bi_source_topic and (inner_target_topic not in stop_words_trigram)):
                trigram_key=bi_source_topic+':'+bi_target_topic+':'+inner_target_topic
                trigram_weight=val*val2
                trigram_weight_conet[trigram_key]=trigram_weight
    
print(trigram_weight_conet)

{'stock:investor:morningstar': 0.2484705882352941, 'stock:investor:financial': 0.31623529411764706, 'stock:investor:market': 0.3783529411764706, 'advice:financial:service': 0.2482161733615222, 'morningstar:stock:fund': 0.267379679144385, 'morningstar:stock:investor': 0.36669213139801377, 'morningstar:stock:risk': 0.2902979373567609, 'morningstar:stock:market': 0.4660045836516425, 'morningstar:investor:stock': 0.3233766233766234, 'morningstar:investor:financial': 0.43636363636363645, 'morningstar:investor:market': 0.5220779220779221, 'morningstar:financial:service': 0.21821202053760191, 'morningstar:global:bank': 0.22146274777853722, 'morningstar:global:financial': 0.24914559125085442, 'morningstar:global:market': 0.2306903622693096, 'morningstar:growth:market': 0.25351834477309765, 'insure:aboriginal:people': 0.24663345004039858, 'insure:customer:bank': 0.18838421444527179, 'insure:funeral_insurance:aboriginal': 0.26995428973277075, 'insure:funeral_insurance:policy': 0.2146624472573839

In [31]:
# Make CSV of trigram co-occurrence net
sourceNode_trig=[]
targetNode_trig=[]
edgeWeight_trig=[]

for key,val in trigram_weight_conet.items():
    composite_topics=key.split(':')
    source=composite_topics[0]+':'+composite_topics[1]
    sourceNode_trig.append(source)
    targetNode_trig.append(composite_topics[2])
    edgeWeight_trig.append(val)
    
tri_cooccur_df=pd.DataFrame()
tri_cooccur_df['source']=sourceNode_trig
tri_cooccur_df['target']=targetNode_trig
tri_cooccur_df['weight']=edgeWeight_trig

tri_cooccur_df.to_csv('trigram_ooccurrence_net.csv',index=False,encoding='utf-8')

In [35]:
# Now work on 4grams:

#trigram_weight_conet=dict()
fourgram_weight_conet=dict()

for key3,val3 in trigram_weight_conet.items():
    composite_topics=key3.split(':')
    tri_source_topic1=composite_topics[0]
    tri_source_topic2=composite_topics[1]
    tri_target_topic=composite_topics[2]
    for key4,val4 in edge_weight_conet.items():
        composite_topics2=key4.split(':')
        inner_source_topic=composite_topics2[0]
        inner_target_topic=composite_topics2[1]
        #print(bi_target_topic , inner_source_topic)
        if (tri_target_topic == inner_source_topic and inner_target_topic != tri_source_topic1 and inner_target_topic != tri_source_topic2 and (inner_target_topic not in stop_words_trigram)):
            fourgram_key=tri_source_topic1+':'+tri_source_topic2+':'+tri_target_topic+':'+inner_target_topic
            fourgram_weight=val3*val4
            if(fourgram_weight > 0.1):
                fourgram_weight_conet[fourgram_key]=fourgram_weight
    
    
print(fourgram_weight_conet)

{'stock:investor:morningstar:fund': 0.10648739495798318, 'stock:investor:morningstar:director': 0.10326050420168068, 'stock:investor:morningstar:management': 0.1387563025210084, 'stock:investor:morningstar:rise': 0.10326050420168068, 'stock:investor:morningstar:financial': 0.12907563025210084, 'stock:investor:morningstar:global': 0.11616806722689076, 'stock:investor:morningstar:growth': 0.10971428571428571, 'stock:investor:morningstar:risk': 0.13552941176470587, 'stock:investor:morningstar:market': 0.20652100840336135, 'stock:investor:financial:service': 0.1328372093023256, 'advice:financial:service:bank': 0.10476656667856456, 'morningstar:stock:investor:financial': 0.20534759358288773, 'morningstar:stock:investor:market': 0.24568372803666924, 'morningstar:investor:stock:fund': 0.13315508021390374, 'morningstar:investor:stock:risk': 0.14456837280366694, 'morningstar:investor:stock:market': 0.23207028265851795, 'morningstar:investor:financial:service': 0.18329809725158566, 'morningstar:

In [39]:
# Make CSV of fourgram co-occurrence net

sourceNode_4g=[]
targetNode_4g=[]
edgeWeight_4g=[]

for key,val in fourgram_weight_conet.items():
    composite_topics=key.split(':')
    source=composite_topics[0]+':'+composite_topics[1]+':'+composite_topics[2]
    sourceNode_4g.append(source)
    targetNode_4g.append(composite_topics[3])
    edgeWeight_4g.append(val)
    
four_cooccur_df=pd.DataFrame()
four_cooccur_df['source']=sourceNode_4g
four_cooccur_df['target']=targetNode_4g
four_cooccur_df['weight']=edgeWeight_4g

four_cooccur_df.to_csv('4gram_ooccurrence_net.csv',index=False,encoding='utf-8')

In [36]:
# Try 5 gram:

#fourgram_weight_conet=dict()
fivegram_weight_conet=dict()

for key5,val5 in fourgram_weight_conet.items():
    composite_topics=key5.split(':')
    four_source_topic1=composite_topics[0]
    four_source_topic2=composite_topics[1]
    four_source_topic3=composite_topics[2]
    four_target_topic=composite_topics[3]
    for key6,val6 in edge_weight_conet.items():
        composite_topics2=key6.split(':')
        inner_source_topic=composite_topics2[0]
        inner_target_topic=composite_topics2[1]
        #print(bi_target_topic , inner_source_topic)
        if (four_target_topic == inner_source_topic and inner_target_topic != four_source_topic1 
            and inner_target_topic != four_source_topic2 and inner_target_topic != four_source_topic3
            and (inner_target_topic not in stop_words_trigram)):
            fivegram_key=four_source_topic1+':'+four_source_topic2+':'+four_source_topic3+':'+four_target_topic+':'+inner_target_topic
            fivegram_weight=val5*val6
            #if(fourgram_weight > 0.1):
            fivegram_weight_conet[fivegram_key]=fivegram_weight
    
    
print(fivegram_weight_conet)

{'stock:investor:morningstar:financial:service': 0.054219269102990034, 'stock:investor:morningstar:global:bank': 0.05502697921273773, 'stock:investor:morningstar:global:financial': 0.061905351614329945, 'stock:investor:morningstar:global:market': 0.05731977001326846, 'stock:investor:morningstar:growth:market': 0.06299185225420967, 'stock:investor:financial:service:bank': 0.056067653276955606, 'morningstar:stock:investor:financial:service': 0.08625792811839325, 'morningstar:investor:financial:service:bank': 0.07736608000878616, 'morningstar:global:financial:service:bank': 0.04417283233584359, 'insure:funeral_insurance:indigenous:aboriginal:people': 0.061421735022396416, 'investor:morningstar:global:financial:service': 0.04604847875502711, 'probe:funeral_insurance:indigenous:aboriginal:people': 0.05521840189780161, 'premium:funeral_insurance:indigenous:aboriginal:people': 0.0557360744155935}


In [38]:
# Make CSV of fivegram co-occurrence net

sourceNode_5g=[]
targetNode_5g=[]
edgeWeight_5g=[]

for key,val in fivegram_weight_conet.items():
    composite_topics=key.split(':')
    source=composite_topics[0]+':'+composite_topics[1]+':'+composite_topics[2]+':'+composite_topics[3]
    sourceNode_5g.append(source)
    targetNode_5g.append(composite_topics[4])
    edgeWeight_5g.append(val)
    
five_cooccur_df=pd.DataFrame()
five_cooccur_df['source']=sourceNode_5g
five_cooccur_df['target']=targetNode_5g
five_cooccur_df['weight']=edgeWeight_5g

five_cooccur_df.to_csv('5gram_ooccurrence_net.csv',index=False,encoding='utf-8')

In [101]:
#import pprint

tstr='0.009*"financial" + 0.007*"insurance" + 0.006*"brexit" + 0.005*"funeral" + 0.005*"people" + 0.005*"acbf" + 0.004*"industry" + 0.004*"aboriginal" + 0.004*"services" + 0.004*"indigenous"'
ltop=[]
t1s=tstr.split('+')
for ctoken in t1s:
    ttoken=ctoken.split('*')
    itok = ttoken[1]
    token=itok.split('"')
    itop=token[1]
    #print(itop)
    ltop.append(np.str(itop))
    
print(ltop)
str1=np.str(ltop)
print(str1)
ndltop=list(set(ltop))
print(ndltop)
#tokens = tokenize_words(ltop)
#print(tokens)
#dtop = Dictionary([str1.split()])
#print("dictionary")
#print(dtop.keys)
#corpus = [id2word.doc2bow(text) for text in ltop]

    

['financial', 'insurance', 'brexit', 'funeral', 'people', 'acbf', 'industry', 'aboriginal', 'services', 'indigenous']
['financial', 'insurance', 'brexit', 'funeral', 'people', 'acbf', 'industry', 'aboriginal', 'services', 'indigenous']
['acbf', 'industry', 'indigenous', 'people', 'aboriginal', 'services', 'insurance', 'financial', 'brexit', 'funeral']
